<a href="https://colab.research.google.com/github/jscienciadados/rule-induction/blob/main/Inducao_de_Regras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Indução de Regras de Risco de Crédito

In [ ]:
!pip install Orange3

In [2]:
import Orange

# Carregando a base de Dados

In [5]:
from Orange.data import Table

In [8]:
base_risco_credito = Table('dados/risco_credito_regras.csv')

In [9]:
base_risco_credito

[[ruim, alta, nenhuma, 0_15 | alto],
 [desconhecida, alta, nenhuma, 15_35 | alto],
 [desconhecida, baixa, nenhuma, 15_35 | moderado],
 [desconhecida, baixa, nenhuma, acima_35 | alto],
 [desconhecida, baixa, nenhuma, acima_35 | baixo],
 ...
]

# Verificando o dominio

In [10]:
base_risco_credito.domain

[historia, divida, garantias, renda | risco]

# Gerando as Regras

In [12]:
cn2 = Orange.classification.rules.CN2Learner()
regras_risco_credito = cn2(base_risco_credito)

In [13]:
for regras in regras_risco_credito.rule_list:
  print(regras)

IF renda==0_15 THEN risco=alto 
IF historia==boa AND divida!=alta THEN risco=baixo 
IF historia==boa AND garantias!=nenhuma THEN risco=baixo 
IF historia==boa AND renda!=15_35 THEN risco=baixo 
IF historia==boa THEN risco=moderado 
IF divida==alta THEN risco=alto 
IF historia!=desconhecida THEN risco=moderado 
IF garantias==adequada THEN risco=baixo 
IF renda==15_35 THEN risco=moderado 
IF historia==desconhecida THEN risco=baixo 
IF TRUE THEN risco=alto 


historia boa, divida alta, garantias nemhuma, renda > 35

historia ruim, divida alta, garantias adequada, renda < 15

##Prevendo o crédito de novos clientes

In [15]:
previsoes = regras_risco_credito([['boa', 'alta', 'nenhuma', 'acima_35'], ['ruim', 'alta', 'adequada', '0_15']])

In [16]:
previsoes

array([1, 0])

Interpretando o Resultado

In [17]:
base_risco_credito.domain.class_var

DiscreteVariable(name='risco', values=('alto', 'baixo', 'moderado'))

In [18]:
for i in previsoes:
  print(base_risco_credito.domain.class_var.values[i])

baixo
alto


##Base Credit Data

In [19]:
base_credit = Orange.data.Table('dados/credit_data_regras.csv')

# Visualizando os dados

In [20]:
base_credit

[[66155.9, 59.017, 8106.53 | 0],
 [34415.2, 48.1172, 6564.75 | 0],
 [57317.2, 63.108, 8020.95 | 0],
 [42709.5, 45.752, 6103.64 | 0],
 [66952.7, 18.5843, 8770.1 | 1],
 ...
]

In [21]:
base_credit.domain

[income, age, loan | default]

# Dividindo os dados em treino e teste

In [22]:
base_dividida = Orange.evaluation.testing.sample(base_credit, n = 0.25)

In [23]:
base_dividida

([[63271.6, 23.3421, 11298.2 | 1],
  [62908.4, 51.1468, 3213.9 | 0],
  [28482.6, 19.7838, 329.45 | 0],
  [34475.2, 42.3079, 3162.13 | 0],
  [26316.8, 37.2177, 923.028 | 0],
  ...
 ], [[27845.8, 51.9706, 4959.92 | 0],
  [43489.8, 51.7338, 6501.04 | 0],
  [56530.5, 45.5449, 5957.99 | 0],
  [53239.5, 63.5151, 4606.16 | 0],
  [29672.6, 51.0198, 607.909 | 0],
  ...
 ])

# Gerando os dados de treinamento e de teste

In [24]:
base_treino = base_dividida[1]
base_teste = base_dividida[0]

In [25]:
print(len(base_treino))
print(len(base_teste))

1500
500


##Gerando as regras com o CN2Learner

In [27]:
cn2 = Orange.classification.rules.CN2Learner()
regras_credit = cn2(base_treino)

In [28]:
for rules in regras_credit.rule_list:
  print(rules)

IF age>=34.9257164876908 THEN default=0 
IF loan<=2495.13299137587 AND income>=20511.4294355735 THEN default=0 
IF loan>=7498.630446855849 AND age>=18.3097456344403 THEN default=1 
IF income>=54748.9123116247 AND income>=56846.4742321521 THEN default=0 
IF loan>=5785.88427485782 AND loan>=6241.27050834808 THEN default=1 
IF income>=48790.1324336417 AND income>=50360.6787929311 THEN default=0 
IF age>=34.7514377613552 THEN default=1 
IF loan>=5785.88427485782 THEN default=1 
IF income<=31702.3342987522 AND age>=34.7410444347188 THEN default=0 
IF income<=31702.3342987522 AND loan>=3665.88089899456 THEN default=1 
IF age>=34.669146894011604 THEN default=1 
IF age>=34.1016539284028 THEN default=0 
IF age>=33.804503521882 THEN default=1 
IF age>=30.676849837880603 AND income>=29724.4768752716 THEN default=0 
IF age>=30.132575557101497 AND loan>=2971.00330971188 THEN default=1 
IF income<=22089.8374845274 AND age>=21.3656869572587 THEN default=1 
IF income>=45971.133494749694 THEN default=0

# Testando o algoritmo

In [29]:
previsoes = Orange.evaluation.testing.TestOnTestData(base_treino, base_teste, [lambda testdata: regras_credit])

In [30]:
previsoes

# Avaliação do desempenho do modelo

In [31]:
Orange.evaluation.CA(previsoes)

array([0.97])